In [57]:
import sqlite3 as sql
import pandas as pd
import os.path
import numpy as np
import mysql.connector

In [58]:
# Create connection to database file
database = "countryclub.db"
conn = sql.connect(database)

In [59]:
os.getcwd()

'C:\\Users\\yuvem\\OneDrive\\Documents\\SQLFiles_Tier2'

In [60]:
import os
path=r"C:\Users\yuvem\OneDrive\Documents\SQLFiles_Tier2"
os.chdir(path)

In [61]:
os.getcwd()

'C:\\Users\\yuvem\\OneDrive\\Documents\\SQLFiles_Tier2'

/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [64]:
query1 = """SELECT *
FROM
(
SELECT 
    f.name,
    SUM(CASE WHEN b.memid !=0 THEN (f.membercost * b.slots) 
        ELSE (f.guestcost * b.slots) END) AS total_revenue
FROM Facilities AS f
LEFT JOIN Bookings AS b ON f.facid = b.facid
GROUP BY f.name
) AS subquery
WHERE total_revenue < 1000
ORDER BY total_revenue;"""

In [65]:
df = pd.read_sql_query(query1, conn)
df.head()

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [66]:
query2 = """SELECT m1.firstname || ' ' || m1.surname AS member_name, m2.firstname || ' ' || m2.surname AS recommender_name
           FROM members AS m1
           JOIN members AS m2
           	ON m1.recommendedby = m2.memid
           WHERE m1.recommendedby = m2.memid
           ORDER BY m1.surname, m1.firstname;"""

In [67]:
df2 = pd.read_sql_query(query2, conn)
df2.head()

,member_name,recommender_name
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith


/* Q12: Find the facilities with their usage by member, but not guests */

In [69]:
query3 = """SELECT f.name, b.memid, m.firstname || ' ' || m.surname AS member_name, COUNT(b.memid) AS COUNT_member_usage
            FROM Bookings AS b
            JOIN Facilities AS f
                ON b.facid = f.facid
            JOIN Members AS m
                ON b.memid = m.memid
            WHERE b.memid != 0
            GROUP BY b.facid, b.memid;"""

In [70]:
df3 = pd.read_sql_query(query3, conn)
df3.head()

,name,memid,member_name,COUNT_member_usage
0,Tennis Court 1,2,Tracy Smith,30
1,Tennis Court 1,3,Tim Rownam,6
2,Tennis Court 1,4,Janice Joplette,19
3,Tennis Court 1,5,Gerald Butters,57
4,Tennis Court 1,6,Burton Tracy,31


/* Q13: Find the facilities usage by month, but not guests */

In [71]:
query4 = """SELECT f.name AS facility_name, strftime('%m', starttime) AS month, COUNT(b.memid) AS COUNT_usage
            FROM Bookings AS b
            JOIN Facilities AS f
                ON b.facid = f.facid
            JOIN Members AS m
                ON b.memid = m.memid
            WHERE b.memid != 0
            GROUP BY b.facid, month;"""

In [72]:
df4 = pd.read_sql_query(query4, conn)
df4.head()

,facility_name,month,COUNT_usage
0,Tennis Court 1,07,65
1,Tennis Court 1,08,111
2,Tennis Court 1,09,132
3,Tennis Court 2,07,41
4,Tennis Court 2,08,109
